# SI 330: Homework 8: Databases & SQL


## Due: Friday, March 23, 2018,  11:59:00pm

### Submission instructions</font>
After completing this homework, you should turn in two files via Canvas ->  Assignments -> HW 7:
Your Notebook, named si330-hw7-YOUR_UNIQUE_NAME.ipynb and
the HTML file, named si330-hw7-YOUR_UNIQUE_NAME.html.

### Name:  Dingan Chen
### Uniqname: Dinganc
### People you worked with: I worked by myself


## Learning Objectives
After completing this Homework, you should know how to:
* create a postgreSQL database on AWS Relational Database Service (RDS)
* load the contents of a CSV file into a pandas DataFrame
* manipulate a pandas DataFrame to extract information from a column
* load the resulting dataframes into a postgreSQL database
* issue SQL queries to answer real-world questions

## Background
We have generated a CSV file that consists of nearly 10,000 books from Goodreads.com.  
The file is derived from a dataset called "goodreads-10k" (see http://fastml.com/goodbooks-10k-a-new-dataset-for-book-recommendations/ for more details; we are only using the books.csv file for this assignment).

The purpose of this assignment is to take that CSV file, manipulate some of the data, and
write the resulting dataset to a postgreSQL database. Whereas this may seem contrived,
it models a real-world scenario in which the data would the be made available via
an API (which may be a nice bonus assignment, don't you think?).

We have identified six (6) steps beyond the initial setup to guide you through this 
assignment.  Each of the following steps is detailed below:
* Step 0: Redefine a function to improve performance and load the required libraries
* Step 1: Fill in the correct  parameters to create a new AWS RDS instance
* Step 2. Read the CSV file
* Step 3. Make a connection to your SQL Database
* Step 4: Breaking out the "authors"
* Step 5: Load your authors table into your postgreSQL database
* Step 6. Query the database

Places where you need to do something are indicated in <font color="magenta">magenta</font>.

### Step 0: Redefine a function to improve performance and load the required libraries

In [1]:
# This must be run before import pandas
# See pandas issue #8953 for an explanation
from pandas.io.sql import SQLTable

def _execute_insert(self, conn, keys, data_iter):
    print("Using monkey-patched _execute_insert")
    data = [dict((k, v) for k, v in zip(keys, row)) for row in data_iter]
    conn.execute(self.insert_statement().values(data))

SQLTable._execute_insert = _execute_insert

# Load required libraries
import pandas as pd
from sqlalchemy import create_engine

### Step 1: Fill in the correct  parameters to create a new AWS RDS instance
You should review Lab 8, log into your AWS Management Console, go to the RDS panel and 
create a new instance called si330wn2018-goodbooks-UNIQNAME (where UNIQNAME is replaced with your uniqname).

Create a database called "goodbooks", with your choice of username and password.

After your database is created, 
** <font color="magenta">record the relevant parameters in the following code block: **


In [2]:
endpoint = "si330wn2018-goodbooks-dinganc.c2uyq5gyfjgu.us-east-1.rds.amazonaws.com"
username = "dinganc"
password = "glazeddonut"

In [3]:
# DO NOT MODIFY THIS
database = "goodbooks" # we asked you to use this name for the database

### Step 2. Read the CSV file

We have provided you the data as a csv file on a public S3 bucket as https://s3.amazonaws.com/si330w18-cteplovs/books.csv. 
The file has four columns, of which you'll only deal with `authors` and `book_id` for this assignment.
You'll need to read the csv file (remember that you can pass a URL to ```pd.read_csv()```)and create a dataframe from it.
<font color="magenta">** Read the csv file and create the DataFrame. The datafile has 9980 rows in it; you should confirm that your dataframe has the correct number of rows. **</font>

In [4]:
books = pd.read_csv('https://s3.amazonaws.com/si330w18-cteplovs/books.csv')
print(len(books))

9980


### Step 3. Make a connection to your SQL Database

In this step we will be using the SQLAlchemy package to connect to and query the database.
SQLAlchemy has many uses.  We're using it to allow us to use the pandas to_sql() function. 
We've provided the code to make the connection assuming that you have updated the variable in Step 1 above.

In [5]:
# This will establish a connection to your database, if you've set everything up properly
engine = create_engine('postgresql://{0}:{1}@{2}:5432/{3}'.format(username,password,endpoint,database))

In the next code block, we are using the pandas method ```.to_sql()``` to write a table to the database. ```.to_sql()``` takes in 3 parameters: the table to be used, the SQLAlchemy connection to the database, and what to do if the table already exists.

In [6]:
books.to_sql('books',engine,if_exists='replace')

Using monkey-patched _execute_insert


### Step 4: Breaking out the "authors"
If you take a look at the books dataframe, you'll see the format of the authors field:
```
3160       James Patterson, Michael Ledwidge
1278               Virgil, Robert Fitzgerald
3106                       Patricia Cornwell
4764                          Scott B. Smith
1207                         Jennifer Probst
9703                        Orson Scott Card
7292                              R.L. Stine
8612                             Bella Andre
8718                             John Irving
7167    Arnaldur Indriðason, Bernard Scudder
```
The author column contains all the authors of a book separated by commas.
While this is useful to a human reader, it makes it very inefficient to 
search for a particular author, especially if the author you're looking
for isn't the first author of the book.

For that reason, we need to extract the author information and
create a new DataFrame called authors.  The authors dataframe should consist of two columns: "author" and "bookid" (use lowercase for the column names -- just trust us on this one).  The dataframe should look like:
```
     author          bookid
0	Suzanne Collins 1
1	J.K. Rowling    2
2	Mary GrandPré   2
3	Stephenie Meyer 3
4	Harper Lee      4
```

<font color="magenta">** Create a pandas DataFrame called authors, based on the contents of the books table.**</font>
Note that you will need to split the authors column on commas and strip whitespace from the resulting strings.

In [7]:
author_list=[]
for ix,value in books.iterrows():
    for author in value['authors'].split(','):
        author_list.append({'author':author.strip(),'bookid':value['id']})
authors=pd.DataFrame.from_dict(author_list)
authors.head()

,author,bookid
0,Suzanne Collins,1
1,J.K. Rowling,2
2,Mary GrandPré,2
3,Stephenie Meyer,3
4,Harper Lee,4


### Step 5: Load your authors table into your postgreSQL database
Assuming you've created a pandas DataFrame called "authors", 
<font color="magenta">** you should be able to 
just run the following code block:**</font>

In [8]:
authors.to_sql('authors',engine,if_exists='replace')

Using monkey-patched _execute_insert


### Step 6. Query the database

Now that you've set everything up, you should answer the following questions **USING SQL and based on the data provided**:

Note: It might be better if you look at some reference on how to make SQL queries with SQLAlchemy. Conceptually and syntactically, it's similar to how you made queries with postgres.

<font color="magenta">**1. Confirm that the number of unique (Hint: DISTINCT) authors is 5829.**</font>

In [9]:
print(len(engine.execute("SELECT DISTINCT author FROM authors").fetchall()))

5829


<font color="magenta">**2.How many books has J.K. Rowling written?**</font>

In [10]:
print(len(engine.execute("SELECT author FROM authors WHERE author LIKE 'J.K. Rowling'").fetchall()))

27


<font color="magenta">**3. Who has co-authored books with J.K. Rowling?**</font>

In [11]:
print([i[0] for i in engine.execute("SELECT DISTINCT author FROM authors WHERE bookid IN (SELECT bookid FROM authors WHERE author LIKE 'J.K. Rowling')").fetchall() if  i[0]!= 'J.K. Rowling'])

['Albus Dumbledore', 'Robert Galbraith', 'Kennilworthy Whisp', 'Jack Thorne', 'Joel Holland', 'Rufus Beck', 'MinaLima', 'Newt Scamander', 'John Tiffany', 'Melissa Anelli', 'Mary GrandPré']


<font color="magenta">**4. Which book has the largest number of authors?  How many authors?  Who are they?**</font>

In [12]:
Q1=engine.execute("SELECT bookid, count(bookid) FROM authors GROUP by bookid ORDER BY count(bookid) DESC LIMIT 1").fetchall()[0]
Q2=[i[0] for i in engine.execute("SELECT author FROM authors WHERE bookid = '6202'").fetchall()]
print ("book {} has {} authors".format(Q1[0],Q1[1]))
print ("which has authors {}".format(', '.join(Q2)))

book 6202 has 47 authors
which has authors Christopher Hitchens, Titus Lucretius Carus, Omar Khayyám, Thomas Hobbes, Baruch Spinoza, David Hume, James Boswell, Percy Bysshe Shelley, George Eliot, Charles Darwin, Leslie Stephen, Anatole France, Mark Twain, Joseph Conrad, Thomas Hardy, Emma Goldman, H.P. Lovecraft, Carl Van Doren, H.L. Mencken, Sigmund Freud, Albert Einstein, George Orwell, John Betjeman, Chapman Cohen, Bertrand Russell, Philip Larkin, Martin Gardner, Carl Sagan, John Updike, John Leslie Mackie, Michael Shermer, A.J. Ayer, Daniel C. Dennett, Charles Templeton, Richard Dawkins, Victor J. Stenger, Elizabeth S.  Anderson, Penn Jillette, Ian McEwan, Steven Weinberg, Salman Rushdie, Ibn Warraq, Sam Harris, A.C. Grayling, Ayaan Hirsi Ali, John Stuart Mill, Karl Marx


### Above and Beyond

Besides the `books.csv` we have also provided two other csv files. You can use them to upload to create tables to your RDS and use it for this section. Make sure that you have used SQL in your above and beyond.

Please also indicate in a separate markdown block why you think your work goes above and beyond.

In [13]:
to_read = pd.read_csv('to_read.csv')
rating = pd.read_csv('ratings.csv')
to_read.to_sql('to_read',engine,if_exists='replace',chunksize=150000)
rating.to_sql('rating',engine,if_exists='replace',chunksize=150000)

Using monkey-patched _execute_insert
Using monkey-patched _execute_insert
Using monkey-patched _execute_insert
Using monkey-patched _execute_insert
Using monkey-patched _execute_insert
Using monkey-patched _execute_insert
Using monkey-patched _execute_insert
Using monkey-patched _execute_insert
Using monkey-patched _execute_insert
Using monkey-patched _execute_insert
Using monkey-patched _execute_insert
Using monkey-patched _execute_insert
Using monkey-patched _execute_insert
Using monkey-patched _execute_insert


In [45]:
Q3=engine.execute("SELECT books.title,AVG(rating.rating) FROM rating LEFT JOIN books on rating.book_id=books.id GROUP by rating.book_id,books.title ORDER BY AVG(rating.rating) DESC LIMIT 10 ").fetchall()
print("top ten highest rated books\n")
for j in [(i[0],round(float(i[1]),2))for i in Q3]:
    print(j[0],'|rating: ',j[1])

top ten highest rated books

ESV Study Bible |rating:  4.82
The Days Are Just Packed: A Calvin and Hobbes Collection |rating:  4.78
The Indispensable Calvin and Hobbes |rating:  4.78
Attack of the Deranged Mutant Killer Monster Snow Goons |rating:  4.78
The Divan |rating:  4.77
There's Treasure Everywhere: A Calvin and Hobbes Collection |rating:  4.77
Harry Potter Boxed Set, Books 1-5 (Harry Potter, #1-5) |rating:  4.77
The Calvin and Hobbes Lazy Sunday Book |rating:  4.75
It's a Magical World: A Calvin and Hobbes Collection |rating:  4.75
The Authoritative Calvin and Hobbes: A Calvin and Hobbes Treasury |rating:  4.75


In [47]:
Q4=engine.execute("SELECT books.title,count(rating.rating) FROM rating LEFT JOIN books on rating.book_id=books.id GROUP by rating.book_id,books.title ORDER BY count(rating.rating) DESC LIMIT 10 ").fetchall()
print("top ten most rated books\n")
for j in [(i[0],int(i[1]))for i in Q4]:
    print(j[0],'|rating count: ',j[1])

top ten most rated books

Harry Potter and the Sorcerer's Stone (Harry Potter, #1) |rating count:  100
Twilight (Twilight, #1) |rating count:  100
To Kill a Mockingbird |rating count:  100
The Great Gatsby |rating count:  100
The Fault in Our Stars |rating count:  100
The Hobbit |rating count:  100
The Catcher in the Rye |rating count:  100
Angels & Demons  (Robert Langdon, #1) |rating count:  100
Pride and Prejudice |rating count:  100
The Hunger Games (The Hunger Games, #1) |rating count:  100


In [50]:
Q5=engine.execute("SELECT books.title, count(to_read.book_id) FROM to_read LEFT JOIN books on to_read.book_id=books.id GROUP by to_read.book_id,books.title ORDER BY count(to_read.book_id) DESC LIMIT 10").fetchall()
print("top ten most wanted books in to read list\n")
for j in [(i[0],int(i[1]))for i in Q5]:
    print(j[0],'|to read count: ',j[1])

top ten most wanted books in to read list

The Book Thief |to read count:  2772
All the Light We Cannot See |to read count:  1967
Catch-22 |to read count:  1840
1984 |to read count:  1812
The Kite Runner |to read count:  1767
Life of Pi |to read count:  1717
Miss Peregrine’s Home for Peculiar Children (Miss Peregrine’s Peculiar Children, #1) |to read count:  1650
A Game of Thrones (A Song of Ice and Fire, #1) |to read count:  1619
Slaughterhouse-Five |to read count:  1608
The Alchemist |to read count:  1576
